# Time Series Prediction with BQML and AutoML

**Objectives**
 1. Learn how to use BQML to create a classification time-series model using `CREATE MODEL`.
 2. Learn how to use BQML to create a linear regression time-series model.
 3. Learn how to use AutoML Tables to build a time series model from data in BigQuery.

## Set up environment variables and load necessary libraries

In [ ]:
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
%env PROJECT = {PROJECT}
%env REGION = "us-central1"

## Create the dataset

In [ ]:
from google.cloud import bigquery
from IPython import get_ipython

bq = bigquery.Client(project=PROJECT)


def create_dataset():
    dataset = bigquery.Dataset(bq.dataset("stock_market"))
    try:
        bq.create_dataset(dataset)  # Will fail if dataset already exists.
        print("Dataset created")
    except:
        print("Dataset already exists")


def create_features_table():
    error = None
    try:
        bq.query(
            """
        CREATE TABLE stock_market.eps_percent_change_sp500
        AS
        SELECT *
        FROM `stock_market.eps_percent_change_sp500`
        """
        ).to_dataframe()
    except Exception as e:
        error = str(e)
    if error is None:
        print("Table created")
    elif "Already Exists" in error:
        print("Table already exists.")
    else:
        print(error)
        raise Exception("Table was not created.")


create_dataset()
create_features_table()

## Review the dataset

In the previous lab we created the data we will use modeling and saved them as tables in BigQuery. Let's examine that table again to see that everything is as we expect. Then, we will build a model using BigQuery ML using this table.

In [ ]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  stock_market.eps_percent_change_sp500
LIMIT
  10

## Using BQML

### Create classification model for `direction`

To create a model
1. Use `CREATE MODEL` and provide a destination table for resulting model. Alternatively we can use `CREATE OR REPLACE MODEL` which allows overwriting an existing model.
2. Use `OPTIONS` to specify the model type (linear_reg or logistic_reg). There are many more options [we could specify](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#model_option_list), such as regularization and learning rate, but we'll accept the defaults.
3. Provide the query which fetches the training data 

Have a look at [Step Two of this tutorial](https://cloud.google.com/bigquery/docs/bigqueryml-natality) to see another example.

**The query will take about two minutes to complete**


We'll start with creating a classification model to predict the `direction` of each stock. 

We'll take a random split using the `symbol` value. With about 500 different values, using `ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1` will give 30 distinct `symbol` values which corresponds to about 171,000 training examples. After taking 70% for training, we will be building a model on about 110,000 training examples.

In [ ]:
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.direction_model OPTIONS(model_type = "logistic_reg",
    input_label_cols = ["direction"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  direction
FROM
  `stock_market.eps_percent_change_sp500`
WHERE
  tomorrow_close IS NOT NULL
  AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
  AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 70

## Get training statistics and examine training info

After creating our model, we can evaluate the performance using the [`ML.EVALUATE` function](https://cloud.google.com/bigquery-ml/docs/bigqueryml-natality#step_four_evaluate_your_model). With this command, we can find the precision, recall, accuracy F1-score and AUC of our classification model.

In [ ]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.direction_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      direction
    FROM
      `stock_market.eps_percent_change_sp500`
    WHERE
      tomorrow_close IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) > 15 * 70
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 85))

We can also examine the training statistics collected by Big Query. To view training results we use the [`ML.TRAINING_INFO`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-train) function.

In [ ]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.direction_model`)
ORDER BY iteration

### Compare to simple benchmark

Another way to asses the performance of our model is to compare with a simple benchmark. We can do this by seeing what kind of accuracy we would get using the naive strategy of just predicted the majority class. For the training dataset, the majority class is 'STAY'. The following query we can see how this naive strategy would perform on the eval set.

In [ ]:
%%bigquery --project $PROJECT
#standardSQL
WITH
  eval_data AS (
  SELECT
    symbol,
    Date,
    Open,
    close_MIN_prior_5_days,
    close_MIN_prior_20_days,
    close_MIN_prior_260_days,
    close_MAX_prior_5_days,
    close_MAX_prior_20_days,
    close_MAX_prior_260_days,
    close_AVG_prior_5_days,
    close_AVG_prior_20_days,
    close_AVG_prior_260_days,
    close_STDDEV_prior_5_days,
    close_STDDEV_prior_20_days,
    close_STDDEV_prior_260_days,
    direction
  FROM
    `stock_market.eps_percent_change_sp500`
  WHERE
    tomorrow_close IS NOT NULL
    AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
    AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) > 15 * 70
    AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 85)
SELECT
  direction,
  (COUNT(direction)* 100 / (
    SELECT
      COUNT(*)
    FROM
      eval_data)) AS percentage
FROM
  eval_data
GROUP BY
  direction

So, the naive strategy of just guessing the majority class would have accuracy of 0.5509 on the eval dataset, just below our BQML model. 

### Create regression model for `normalized change`

We can also use BigQuery to train a regression model to predict the normalized change for each stock. To do this in BigQuery we need only change the OPTIONS when calling `CREATE OR REPLACE MODEL`. This will give us a more precise prediction rather than just predicting if the stock will go up, down, or stay the same. Thus, we can treat this problem as either a regression problem or a classification problem, depending on the business needs.


In [ ]:
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.price_model OPTIONS(model_type = "linear_reg",
    input_label_cols = ["normalized_change"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  normalized_change
FROM
  `stock_market.eps_percent_change_sp500`
WHERE
  normalized_change IS NOT NULL
  AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
  AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 70

Just as before we can examine the evaluation metrics for our regression model and examine the training statistics in Big Query

In [ ]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.price_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      normalized_change
    FROM
      `stock_market.eps_percent_change_sp500`
    WHERE
      normalized_change IS NOT NULL
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15)) = 1
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) > 15 * 70
      AND ABS(MOD(FARM_FINGERPRINT(symbol), 15 * 100)) <= 15 * 85))

In [ ]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.price_model`)
ORDER BY iteration

## Train a Time Series model using AutoML



### Step 1. Create a Dataset

In the GCP Navigation Menu, navigate to **Vertex AI -> Datasets**.

Select **Create** and give it a name like `stock_market`. In the **Select a data type and objective** section, select **Tabular** and under Tabular select, **Regression/Classification**. Click **Create** at the bottom.

<img src='../assets/create_dataset.png' width='70%'>

### Step 2. Import the Data

Once you have created the dataset, you can add data to it using BigQuery. Under **Select a data source** select **Select a table or view from BigQuery**. In the BigQuery path field, browse for `eps_percent_change_sp500`.

<img src='../assets/import_data.png' width='50%'>

### Step 3. Train the model 

Once the data has been imported into the dataset. You can get basic statistics for each of the feature columns, by clicking **GENERATE STATISTICS** (Note: This will take a couple of minutes). Since the data has been imported we can now proceed and train a classification model. Select **Train a new model** from the left column. Set the objective as "Classification". Ensure that the model training method is set to AutoML. Click **Continue**

<img src='../assets/set_training_objective.png' width='80%'>

In Model details, select the column `direction` as the Target Column. This is the label column we want to train our model on. Click **Continue**

<img src='../assets/select_target_label.png' width='80%'>

In Training options, exclude all columns **except** for the following
```
symbol
Date
Open
close_MIN_prior_5_days
close_MIN_prior_20_days
close_MIN_prior_260_days
close_MAX_prior_5_days
close_MAX_prior_20_days
close_MAX_prior_260_days
close_AVG_prior_5_days
close_AVG_prior_20_days
close_AVG_prior_260_days
close_STDDEV_prior_5_days
close_STDDEV_prior_20_days
close_STDDEV_prior_260_days
```
These are the same columns we have used in our BQML model. This will help us compare the performance between BQML models and AutoML models.
<img src='../assets/select_training_columns.png' width='80%'>

In Compute and pricing, set the **Budget** as 1. We get a decent performance for our model using just one node hour of training. Having **Enable early stopping** selected ensures that the model training will stop if there is no improvement. Now, you can select **START TRAINING**. Your model should start training.

<img src='../assets/select_node_hours.png' width='80%'>

### Step 4. Evaluate your model.

Training can take many hours. But once training is complete you can inspect the evaluation metrics of your model. Since this is a classification task, we can also adjust the threshold and explore how different thresholds will affect your evaluation metrics. Also on that page, we can explore the feature importance of the various features used in the model and view confusion matrix for our model predictions.

<img src='../assets/analyze_eval_metrics.png' width='80%'>

### Step 5. Predict with the trained model. 

Once the model is done training, navigate to the `Deploy and test` tab and Deploy the model to an endpoint, so we can test prediction. 

<img src='../assets/deploy_model_endpoint.png' width='80%'>

When calling predictions, you can call batch prediction jobs by specifying a BigQuery table or csv file. Or you can do online prediction for a single instance.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License